# Feature Creation Methods
Call these methods to add features to the Flights Training dataframe

In [6]:
from datetime import datetime
import pandas as pd
import pickle

## 0a. TRAINING: Read from the Flights 2019 table

In [7]:
filename = 'flight_2019'

infile = open(filename,'rb')
flight_2019 = pickle.load(infile)
infile.close()

#### New dataframe with a smaller sample of 1,000,000 records

In [8]:
flight_sample = flight_2019.sample(n=1000000, random_state=1)

In [9]:
flight_sample.shape

(1000000, 38)

## 0b. TESTING: Read from the Flights January 2020 table

In [60]:
%run ./db_connection.ipynb

In [76]:
flight_test = """
SELECT TO_DATE('2017-01-03','YYYY-MM-DD'),
    fl_date,
    TO_DATE(fl_date, 'YYYY-MM-DD'),
    ate_part('month', fl_date)
    from flights_test
    limit 10

; """

flight_test_2020 = """
    SELECT 
    fl_date,
    mkt_unique_carrier,
    branded_code_share,
    mkt_carrier,
    mkt_carrier_fl_num,
    op_unique_carrier,
    tail_num,
    op_carrier_fl_num,
    origin_airport_id,
    origin,
    origin_city_name,
    dest_airport_id,
    dest,
    dest_city_name,
    crs_dep_time,
    crs_arr_time,
    dup,
    crs_elapsed_time,
    flights,
    distance    
    FROM flights_test
    where date_part('day', fl_date) < 8
    
"""
flight_sample = execute_read_query(con, flight_test_2020)

In [77]:
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


In [78]:
flight_sample.shape

(150623, 20)

## 1. Is Flight Delayed Feature (0 or 1 values)

In [10]:
# def is_delayed_feature(flights_df):
#     flights_df_cp = flights_df.copy()
    
#     flights_df_cp['dep_delay'] = flights_df_cp['dep_delay'].fillna(0)
#     flights_df_cp['arr_delay'] = flights_df_cp['arr_delay'].fillna(0)
#     flights_df_cp['carrier_delay'] = flights_df_cp['carrier_delay'].fillna(0)
#     flights_df_cp['weather_delay'] = flights_df_cp['weather_delay'].fillna(0)
#     flights_df_cp['nas_delay'] = flights_df_cp['nas_delay'].fillna(0)
#     flights_df_cp['security_delay'] = flights_df_cp['security_delay'].fillna(0)
#     flights_df_cp['late_aircraft_delay'] = flights_df_cp['late_aircraft_delay'].fillna(0)

#     flights_df_cp['total_delay'] = flights_df_cp.apply(
#         lambda x: ((x['dep_delay'] + x['arr_delay'] +
#                  x['carrier_delay'] +
#                  x['weather_delay'] + x['nas_delay'] + x['security_delay'] +
#                  x['late_aircraft_delay']
#         )),  axis=1)
    
#     return flights_df_cp.apply(
#         lambda x: (0 if x['total_delay'] <= 0 else 1),  axis=1)
    

In [11]:
# flight_sample['is_delayed'] = is_delayed_feature(flight_sample)
# flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,fl_date,year,month,is_delayed
5817519,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,3.0,0.0,108.0,None,NaN,NaN,2019-05-23,2019.0,5.0,1
3440584,DL,DL_CODESHARE,4566,OO,N477CA,4566,10397,12007,2019,2017.0,...,NaN,NaN,NaN,None,NaN,NaN,2019-12-05,2019.0,12.0,0
2213756,DL,DL,2151,DL,N844DN,2151,10721,13487,1820,1802.0,...,NaN,NaN,NaN,None,NaN,NaN,2019-02-06,2019.0,2.0,0
4372882,AA,AA,439,AA,N155NN,439,13930,14107,705,656.0,...,NaN,NaN,NaN,None,NaN,NaN,2019-09-17,2019.0,9.0,0
2224046,NK,NK,318,NK,N907NK,318,14635,13487,1315,1311.0,...,NaN,NaN,NaN,None,NaN,NaN,2019-02-06,2019.0,2.0,0


## 2. Flight Speed

In [79]:
def flight_speed_feature(flights_df):
    flights_df_cp = flights_df.copy()

    if ("cancelled" in flights_df):
        flights_df_cp['distance'] = flights_df_cp.apply(
            lambda x: (0.0 if ((x['cancelled'] == 1) | (x['diverted'] == 1)) else x['distance']
                  ),  axis=1)
    
        flights_df_cp['crs_elapsed_time'] = flights_df_cp.apply(
            lambda x: (60.0 if ((x['cancelled'] == 1) | (x['diverted'] == 1)) else x['crs_elapsed_time']
                  ),  axis=1)
    
    return flights_df_cp.apply(
        lambda x: ((x['distance'] / (x['crs_elapsed_time']/60)
              )),  axis=1)

In [80]:
flight_sample['flight_speed'] = flight_speed_feature(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,flight_speed
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363,229.263158
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,...,14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363,242.000000
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,...,14831,SJC,"San Jose, CA",2020,2130,N,70,1,333,285.428571
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,...,14831,SJC,"San Jose, CA",1340,1455,N,75,1,333,266.400000
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,...,14831,SJC,"San Jose, CA",915,1035,N,80,1,333,249.750000


## 3. Flight Haul Type (values of 0, 1 and 2 for Short, Medium and Long)

In [81]:
def flight_haul_type_feature(flights_df):
    flights_df_cp = flights_df.copy()
    
    if ("cancelled" in flights_df):
        flights_df_cp['crs_elapsed_time'] = flights_df_cp.apply(
            lambda x: (0 if ((x['cancelled'] == 1) | (x['diverted'] == 1)) else x['crs_elapsed_time']
                  ),  axis=1)
    
    return flights_df_cp.apply(
        lambda x: 0 if x['crs_elapsed_time'] <= 180.0 else (2 if x['crs_elapsed_time'] >= 360.0 else 1), axis=1
)

In [82]:
flight_sample['flight_haul_type'] = flight_haul_type_feature(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,flight_speed,flight_haul_type
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,SFO,"San Francisco, CA",1810,1945,N,95,1,363,229.263158,0
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,...,SFO,"San Francisco, CA",1150,1320,N,90,1,363,242.000000,0
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,...,SJC,"San Jose, CA",2020,2130,N,70,1,333,285.428571,0
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,...,SJC,"San Jose, CA",1340,1455,N,75,1,333,266.400000,0
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,...,SJC,"San Jose, CA",915,1035,N,80,1,333,249.750000,0


## 4a. TRAINING Month of the Year (1 to 12)

In [83]:
def month_feature(flights_df):
    flights_df_cp = flights_df.copy()
    
    return flights_df_cp[flights_df_cp['fl_date'].notna()].apply(
        lambda x: datetime.strptime(x['fl_date'], '%Y-%m-%d').date().month, axis=1)

## 4b TEST Month of the Year

In [91]:
def month_feature(flights_df):
    flights_df_cp = flights_df.copy()
    
    return flights_df_cp[flights_df_cp['fl_date'].notna()].apply(
        lambda x: (x['fl_date'].month), axis=1)

In [92]:
flight_sample['flight_month'] = month_feature(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,flight_speed,flight_haul_type,flight_month
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,"San Francisco, CA",1810,1945,N,95,1,363,229.263158,0,1
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,...,"San Francisco, CA",1150,1320,N,90,1,363,242.000000,0,1
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,...,"San Jose, CA",2020,2130,N,70,1,333,285.428571,0,1
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,...,"San Jose, CA",1340,1455,N,75,1,333,266.400000,0,1
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,...,"San Jose, CA",915,1035,N,80,1,333,249.750000,0,1


## 5. Arrival Hour of the Day (0 to 23)

In [93]:
def arrival_hour_of_day_feature(flights_df):
    flights_df_cp = flights_df.copy()
    
    return flights_df_cp[flights_df_cp['crs_arr_time'].notna()].apply(
        lambda x: 0 if x['crs_arr_time'] == 2400.0 else datetime.strptime(str(int(x['crs_arr_time'])).zfill(4), '%H%M').time().hour, axis=1)
    

In [94]:
flight_sample['arrival_hour_of_day'] = arrival_hour_of_day_feature(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,1810,1945,N,95,1,363,229.263158,0,1,19
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,...,1150,1320,N,90,1,363,242.000000,0,1,13
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,...,2020,2130,N,70,1,333,285.428571,0,1,21
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,...,1340,1455,N,75,1,333,266.400000,0,1,14
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,...,915,1035,N,80,1,333,249.750000,0,1,10


## 6. Departure Hour of the Day (0 to 23)

In [95]:
def departure_hour_of_day_feature(flights_df):
    flights_df_cp = flights_df.copy()
    
    return flights_df_cp[flights_df_cp['crs_dep_time'].notna()].apply(
        lambda x: 0 if x['crs_dep_time'] == 2400.0 else datetime.strptime(str(int(x['crs_dep_time'])).zfill(4), '%H%M').time().hour, axis=1)
  

In [96]:
flight_sample['departure_hour_of_day'] = departure_hour_of_day_feature(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,crs_arr_time,dup,crs_elapsed_time,flights,distance,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,1945,N,95,1,363,229.263158,0,1,19,18
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,...,1320,N,90,1,363,242.000000,0,1,13,11
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,...,2130,N,70,1,333,285.428571,0,1,21,20
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,...,1455,N,75,1,333,266.400000,0,1,14,13
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,...,1035,N,80,1,333,249.750000,0,1,10,9


## 7. Percent of Flights Delay by Month

In [115]:
#flight_sample = flight_sample.drop(columns='arr_month')
#flight_sample = flight_sample.drop(columns=['percent_delay_x','percent_delay_y'])

In [97]:
def percent_delayed_by_month(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('percent_delay' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['percent_delay'])
        
    # Load the delay by month dataframe
    filename = './reference/percent_delay_by_month_2019'

    infile = open(filename,'rb')
    delay_df = pickle.load(infile)
    infile.close()
    
    # Join flights_df to delay_df, and return only the percent_delay column
    delay_join = pd.merge(flights_df_cp, 
                 delay_df,
                 how="inner",
                 left_on=['origin_airport_id','flight_month'],
                 right_on=['origin_airport_id','arr_month'])
    delay_join = delay_join.drop(columns='arr_month')
    return delay_join


In [98]:
flight_sample = percent_delayed_by_month(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dup,crs_elapsed_time,flights,distance,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,N,95,1,363,229.263158,0,1,19,18,0.000165
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,...,N,90,1,363,242.000000,0,1,13,11,0.000165
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,...,N,70,1,333,285.428571,0,1,21,20,0.000165
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,...,N,75,1,333,266.400000,0,1,14,13,0.000165
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,...,N,80,1,333,249.750000,0,1,10,9,0.000165


## 8. Percent of Flights per Hour

In [99]:
def percent_flights_per_hour(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('percent_flights' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['percent_flights'])
    
    
    filename = './reference/percent_flights_by_hour_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id','arrival_hour_of_day'],
                 right_on=['origin_airport_id','hour_of_day'])
    
    local_join = local_join.drop(columns='hour_of_day')
    
    return local_join

In [100]:
flight_sample = percent_flights_per_hour(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,crs_elapsed_time,flights,distance,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,95,1,363,229.263158,0,1,19,18,0.000165,0.000169
1,2020-01-01,WN,WN,WN,5722,WN,N7820L,5722,13891,ONT,...,80,1,390,292.500000,0,1,19,18,0.000165,0.000169
2,2020-01-01,AA,AA,AA,1086,AA,N967AN,1086,13891,ONT,...,175,1,1188,407.314286,0,1,19,14,0.000165,0.000169
3,2020-01-01,WN,WN,WN,1499,WN,N716SW,1499,13891,ONT,...,75,1,325,260.000000,0,1,19,17,0.000165,0.000169
4,2020-01-02,WN,WN,WN,2966,WN,N400WN,2966,13891,ONT,...,130,1,819,378.000000,0,1,19,15,0.000165,0.000169


## 9. Average Flights per Airport

In [101]:
def average_flights_per_airport(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('avg_flights' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['avg_flights'])
    
    filename = './reference/average_flights_by_airport_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id'],
                 right_on=['origin_airport_id'])
    return local_join

In [102]:
flight_sample = average_flights_per_airport(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,flights,distance,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,1,363,229.263158,0,1,19,18,0.000165,0.000169,236.0
1,2020-01-01,WN,WN,WN,5722,WN,N7820L,5722,13891,ONT,...,1,390,292.500000,0,1,19,18,0.000165,0.000169,236.0
2,2020-01-01,AA,AA,AA,1086,AA,N967AN,1086,13891,ONT,...,1,1188,407.314286,0,1,19,14,0.000165,0.000169,236.0
3,2020-01-01,WN,WN,WN,1499,WN,N716SW,1499,13891,ONT,...,1,325,260.000000,0,1,19,17,0.000165,0.000169,236.0
4,2020-01-02,WN,WN,WN,2966,WN,N400WN,2966,13891,ONT,...,1,819,378.000000,0,1,19,15,0.000165,0.000169,236.0


## 10. Average Passengers By Month

In [103]:
def average_passengers_by_month(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('avg_passengers' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['avg_passengers'])
    
    filename = './reference/average_passengers_by_airport_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id'],
                 right_on=['origin_airport_id'])
    return local_join

In [104]:
flight_sample = average_passengers_by_month(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights,avg_passengers
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,363,229.263158,0,1,19,18,0.000165,0.000169,236.0,151.0
1,2020-01-01,WN,WN,WN,5722,WN,N7820L,5722,13891,ONT,...,390,292.500000,0,1,19,18,0.000165,0.000169,236.0,151.0
2,2020-01-01,AA,AA,AA,1086,AA,N967AN,1086,13891,ONT,...,1188,407.314286,0,1,19,14,0.000165,0.000169,236.0,151.0
3,2020-01-01,WN,WN,WN,1499,WN,N716SW,1499,13891,ONT,...,325,260.000000,0,1,19,17,0.000165,0.000169,236.0,151.0
4,2020-01-02,WN,WN,WN,2966,WN,N400WN,2966,13891,ONT,...,819,378.000000,0,1,19,15,0.000165,0.000169,236.0,151.0


## 11. Total Distance By Month

In [105]:
def total_distance_by_month(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('total_distance' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['total_distance'])
    
    
    filename = './reference/total_distance_per_month_by_airport_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id','flight_month'],
                 right_on=['origin_airport_id','arr_month'])
    
    local_join = local_join.drop(columns='arr_month')
    
    return local_join

In [106]:
flight_sample = total_distance_by_month(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,flight_speed,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights,avg_passengers,total_distance
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,229.263158,0,1,19,18,0.000165,0.000169,236.0,151.0,132805.0
1,2020-01-01,WN,WN,WN,5722,WN,N7820L,5722,13891,ONT,...,292.500000,0,1,19,18,0.000165,0.000169,236.0,151.0,132805.0
2,2020-01-01,AA,AA,AA,1086,AA,N967AN,1086,13891,ONT,...,407.314286,0,1,19,14,0.000165,0.000169,236.0,151.0,132805.0
3,2020-01-01,WN,WN,WN,1499,WN,N716SW,1499,13891,ONT,...,260.000000,0,1,19,17,0.000165,0.000169,236.0,151.0,132805.0
4,2020-01-02,WN,WN,WN,2966,WN,N400WN,2966,13891,ONT,...,378.000000,0,1,19,15,0.000165,0.000169,236.0,151.0,132805.0


## 12. Total Passengers By Month

In [107]:
flight_sample = flight_sample.drop(columns=['month'])

KeyError: "['month'] not found in axis"

In [108]:
def total_passengers_by_month(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('total_passengers' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['total_passengers'])
    
    
    filename = './reference/total_passengers_by_airport_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id','flight_month'],
                 right_on=['origin_airport_id','month'])
    #print(local_join.columns)
    local_join = local_join.drop(columns='month')
    
    return local_join

In [109]:
flight_sample = total_passengers_by_month(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,flight_haul_type,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights,avg_passengers,total_distance,total_passengers
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,0,1,19,18,0.000165,0.000169,236.0,151.0,132805.0,172
1,2020-01-01,WN,WN,WN,5722,WN,N7820L,5722,13891,ONT,...,0,1,19,18,0.000165,0.000169,236.0,151.0,132805.0,172
2,2020-01-01,AA,AA,AA,1086,AA,N967AN,1086,13891,ONT,...,0,1,19,14,0.000165,0.000169,236.0,151.0,132805.0,172
3,2020-01-01,WN,WN,WN,1499,WN,N716SW,1499,13891,ONT,...,0,1,19,17,0.000165,0.000169,236.0,151.0,132805.0,172
4,2020-01-02,WN,WN,WN,2966,WN,N400WN,2966,13891,ONT,...,0,1,19,15,0.000165,0.000169,236.0,151.0,132805.0,172


## 13. Average Fuel by Carrier

In [110]:
def average_fuel(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('avg_fuel' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['avg_fuel'])
    
    
    filename = './reference/average_fuel_by_carrier_2019'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['op_unique_carrier'],
                 right_on=['unique_carrier'])

    local_join = local_join.drop(columns='unique_carrier')
    
    return local_join

In [111]:
flight_sample = average_fuel(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,flight_month,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights,avg_passengers,total_distance,total_passengers,avg_fuel
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,1,19,18,0.000165,0.000169,236.0,151.0,132805.0,172,176971032.0
1,2020-01-01,WN,WN,WN,5722,WN,N7820L,5722,13891,ONT,...,1,19,18,0.000165,0.000169,236.0,151.0,132805.0,172,176971032.0
2,2020-01-01,WN,WN,WN,1499,WN,N716SW,1499,13891,ONT,...,1,19,17,0.000165,0.000169,236.0,151.0,132805.0,172,176971032.0
3,2020-01-02,WN,WN,WN,2966,WN,N400WN,2966,13891,ONT,...,1,19,15,0.000165,0.000169,236.0,151.0,132805.0,172,176971032.0
4,2020-01-02,WN,WN,WN,2047,WN,N248WN,2047,13891,ONT,...,1,19,18,0.000165,0.000169,236.0,151.0,132805.0,172,176971032.0


In [47]:
flight_sample.shape

(952618, 50)

## 14. Add airport State

In [112]:
def origin_aiport_state(flights_df):
    flights_df_cp = flights_df.copy()
    
    # make re-runnable
    if ('origin_state' in flights_df_cp.columns):
        flights_df_cp = flights_df_cp.drop(columns=['origin_state'])
    
    
    filename = './reference/airport_state_mapping'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['origin_airport_id'],
                 right_on=['airport_id'])

    local_join = local_join.drop(columns='airport_id')
    
    return local_join

In [113]:
flight_sample = origin_aiport_state(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,arrival_hour_of_day,departure_hour_of_day,percent_delay,percent_flights,avg_flights,avg_passengers,total_distance,total_passengers,avg_fuel,state
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,19,18,0.000165,0.000169,236.0,151.0,132805.0,172,176971032.0,CA
1,2020-01-01,WN,WN,WN,5722,WN,N7820L,5722,13891,ONT,...,19,18,0.000165,0.000169,236.0,151.0,132805.0,172,176971032.0,CA
2,2020-01-01,WN,WN,WN,1499,WN,N716SW,1499,13891,ONT,...,19,17,0.000165,0.000169,236.0,151.0,132805.0,172,176971032.0,CA
3,2020-01-02,WN,WN,WN,2966,WN,N400WN,2966,13891,ONT,...,19,15,0.000165,0.000169,236.0,151.0,132805.0,172,176971032.0,CA
4,2020-01-02,WN,WN,WN,2047,WN,N248WN,2047,13891,ONT,...,19,18,0.000165,0.000169,236.0,151.0,132805.0,172,176971032.0,CA


## 15. Weather by State and Month

In [114]:
def weather_by_state(flights_df):
    flights_df_cp = flights_df.copy()
    
    filename = './reference/weather_by_month_2020_01'

    infile = open(filename,'rb')
    local_df = pickle.load(infile)
    infile.close()
    
    local_join = pd.merge(flights_df_cp, 
                 local_df,
                 how="inner",
                 left_on=['state','flight_month'],
                 right_on=['state','month'])

    local_join = local_join.drop(columns='month')
    
    return local_join

In [115]:
flight_sample = weather_by_state(flight_sample)
flight_sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,avg_passengers,total_distance,total_passengers,avg_fuel,state,avgCloud,avgDryDays,avgRainDays,avgSnowDays,avgThunderDays
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,...,151.0,132805.0,172,176971032.0,CA,25.037624,23,6,0,1
1,2020-01-01,WN,WN,WN,5722,WN,N7820L,5722,13891,ONT,...,151.0,132805.0,172,176971032.0,CA,25.037624,23,6,0,1
2,2020-01-01,WN,WN,WN,1499,WN,N716SW,1499,13891,ONT,...,151.0,132805.0,172,176971032.0,CA,25.037624,23,6,0,1
3,2020-01-02,WN,WN,WN,2966,WN,N400WN,2966,13891,ONT,...,151.0,132805.0,172,176971032.0,CA,25.037624,23,6,0,1
4,2020-01-02,WN,WN,WN,2047,WN,N248WN,2047,13891,ONT,...,151.0,132805.0,172,176971032.0,CA,25.037624,23,6,0,1


In [117]:
flight_sample.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
       'flight_speed', 'flight_haul_type', 'flight_month',
       'arrival_hour_of_day', 'departure_hour_of_day', 'percent_delay',
       'percent_flights', 'avg_flights', 'avg_passengers', 'total_distance',
       'total_passengers', 'avg_fuel', 'state', 'avgCloud', 'avgDryDays',
       'avgRainDays', 'avgSnowDays', 'avgThunderDays'],
      dtype='object')

In [116]:
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
flight_sample.to_csv('flights_test_2020.zip', index=False,
          compression=compression_opts)  

## 16. Weather by Month

In [55]:
def weather_patterns(flights_df):
    flights_df_cp = flights_df.copy()
    
    weather_pattern_df = pd.read_csv('./reference/Flight_Pattern_Data.csv', index_col=0)
    weather_pattern_df

    
    local_join = pd.merge(flights_df_cp, 
                 weather_pattern_df,
                 how="inner",
                 left_on=['flight_month'],
                 right_on=['month'])

    local_join = local_join.drop(columns='month')
    
    return local_join



In [56]:
flight_sample = weather_patterns(flight_sample)
flight_sample.head()

,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,...,early_count,early_avgDryDays,early_avgRainDays,early_avgCloud,early_avgSnowDays,delay_count,delay_avgDryDays,delay_avgRainDays,delay_avgCloud,delay_avgSnowDays
0,DL,DL_CODESHARE,4016,OO,N291SY,4016,12953,13342,1747,1944.0,...,55.5,17,11,3,0,30.8,18,10,3,0
1,DL,DL_CODESHARE,4016,OO,N292SY,4016,12953,13342,1747,1929.0,...,55.5,17,11,3,0,30.8,18,10,3,0
2,DL,DL_CODESHARE,4016,OO,N278SY,4016,12953,13342,1747,1750.0,...,55.5,17,11,3,0,30.8,18,10,3,0
3,DL,DL_CODESHARE,3616,OO,N268SY,3616,12953,11995,1729,1851.0,...,55.5,17,11,3,0,30.8,18,10,3,0
4,DL,DL_CODESHARE,3607,OO,N276SY,3607,12953,11995,910,906.0,...,55.5,17,11,3,0,30.8,18,10,3,0


# Export Sample Data

In [57]:
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
flight_sample.to_csv('flight_sample1.zip', index=False,
          compression=compression_opts)  